# Clean data

Normalization and imputing missing values.

In [1]:
import pandas as pd
import numpy as np

## Load data

In [2]:
X = pd.read_csv("inputs.tsv", sep="\t", index_col=0)
X.isna().sum(axis=0) / X.shape[0]

cancer_type                            0.000000
tumor_normal_residuals_corr            0.000000
prot_RNA_tumor_normal_ratios_corr      0.000000
above_reg_line_ACSS1_Normal            0.000000
above_reg_line_ACSS1_Tumor             0.000000
above_reg_line_C1orf116_Normal         0.417178
above_reg_line_C1orf116_Tumor          0.417178
above_reg_line_CARD9_Normal            0.288344
above_reg_line_CARD9_Tumor             0.288344
above_reg_line_CDKN2A_Normal           0.230061
above_reg_line_CDKN2A_Tumor            0.230061
above_reg_line_CFI_Normal              0.957055
above_reg_line_CFI_Tumor               0.957055
above_reg_line_DYNC1LI1_Normal         0.539877
above_reg_line_DYNC1LI1_Tumor          0.539877
above_reg_line_ECH1_Normal             0.128834
above_reg_line_ECH1_Tumor              0.128834
above_reg_line_ECT2_Normal             0.000000
above_reg_line_ECT2_Tumor              0.000000
above_reg_line_FAM57A_Normal           0.481595
above_reg_line_FAM57A_Tumor            0

In [3]:
y = pd.read_csv("targets.tsv", sep="\t", index_col=0)
for col in y.columns:
    print(col)
    print(y[col].value_counts(dropna=False).sort_index())
    print()

Recurrence status (1, yes; 0, no)
0.0    265
1.0     61
Name: Recurrence status (1, yes; 0, no), dtype: int64

Survial status (1, dead; 0, alive)
0.0    260
1.0     57
NaN      9
Name: Survial status (1, dead; 0, alive), dtype: int64

histologic_grade
G1 Well differentiated                                                                            25
G1: Nuclei round, uniform, approximately 10µm; nucleoli inconspicuous                              3
G2 Moderately differentiated                                                                     135
G2: Nuclei slightly irregular, approximately 15µm; nucleoli evident                               35
G3 Poorly differentiated                                                                          89
G3: Nuclei very irregular, approximately 20µm; nucleoli large and prominent                       28
G4: Nuclei bizarre and multilobulated, 20µm or greater; nucleoli prominent, chromatin clumped      9
GX Grade cannot be assessed              

## Input: Impute unknowns and normalize

### Split table

Since different column types require different operations, we're going to split the dataframe based on column type.

In [4]:
cat_cols = ["cancer_type"] # Categorical
bool_cols = X.columns[X.columns.str.startswith("above_reg_line_")] # Boolean
num_cols = X.columns[X.columns.str.startswith("tumor_normal_residual_diff_")].tolist() + [
    "tumor_normal_residuals_corr", "prot_RNA_tumor_normal_ratios_corr"
] # Numerical

In [5]:
X_cat = X[cat_cols].copy()
X_bool = X[bool_cols].copy()
X_num = X[num_cols].copy()

### Categorical column is just cancer type, doesn't need any work

In [6]:
X_cat.value_counts(dropna=False)

cancer_type
luad           101
lscc            94
ccrcc           75
hnscc           42
endometrial     14
dtype: int64

### Impute boolean column NaNs

Fill NaNs in boolean columns with a random array of trues and falses, with the same proportion of trues and falses as the original non-NaN cells.

In [7]:
# Select column with tied mode
for col in bool_cols:
    
    # Get count of nans to fill
    nan_ct = X_bool[col].isna().sum()
    
    # Find the proportion of trues and falses in the original non-NaN values
    true_prop = X_bool[col].mean()
    
    # Create an array that size with the same proportion of trues and falses
    true_ct = round(true_prop * nan_ct)
    false_ct = nan_ct - true_ct
    fill_arr = np.append(np.full(true_ct, True), np.full(false_ct, False))
    
    # Shuffle the array
    rng = np.random.default_rng(0)
    rng.shuffle(fill_arr)
    
    # Assign to the column
    X_bool.loc[X_bool[col].isna(), col] = fill_arr

### Impute numerical column NaNs

We'll just fill with the column mean.

In [8]:
X_num = X_num.fillna(X_num.mean(axis=0))

### Normalize numerical columns

In [9]:
mins = X_num.min(axis=0)
maxs = X_num.max(axis=0)
X_num = (X_num - mins) / (maxs - mins)

### Join tables and save

In [10]:
X_joined = X_cat.join(X_bool).join(X_num)
X_joined.to_csv("inputs_cleaned.tsv", sep="\t")

## Targets: Impute NaNs

We'll just fill with the column mode.

In [11]:
y = y.fillna(y.mode(dropna=True).iloc[0, :])
    

Also, we're going to combine the similar groups in histologic_grade.

In [12]:
y = y.assign(histologic_grade=y["histologic_grade"].str[:2])

Check the finished product.

In [13]:
for col in y.columns:
    print(col)
    print(y[col].value_counts(dropna=False).sort_index())
    print()

Recurrence status (1, yes; 0, no)
0.0    265
1.0     61
Name: Recurrence status (1, yes; 0, no), dtype: int64

Survial status (1, dead; 0, alive)
0.0    269
1.0     57
Name: Survial status (1, dead; 0, alive), dtype: int64

histologic_grade
G1     28
G2    170
G3    117
G4      9
GX      2
Name: histologic_grade, dtype: int64

histologic_type
Acinar adenocarcinoma                                        24
Acinar adenocarcinoma and papillary adenocarcinoma            1
Adenocarcinoma                                               53
Adenocarcinoma, acinar predominant                            1
Adenocarcinoma, micropapillary and acinar                     1
Adenocarcinoma, mixed subtype                                 1
Adenocaricnoma, mixed sub-type (solid and acinar)             1
Adenosquamous carcinoma                                       3
Adenosquamous carcinoma (approx. 75% adeno, 25% squamous)     1
Basaloid squamous cell carcinoma                              1
Clear cell rena

Save the targets.

In [14]:
y.to_csv("targets_cleaned.tsv", sep="\t")